In [16]:
import os
import re
import json

import torch
import pandas as pd
from tqdm.auto import tqdm

from datasets import Dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from trl import DPOConfig, DPOTrainer

## 加载数据集

使用预先提供的数据集，其中包括带标签的偏好数据集（labelled_data.json）和测试提示数据（test_prompt.json）。改数据集讨论是否 “真人化动漫”，其中两个回答分别对应支持和不支持（由chatgpt生成），在后面的代码我们需要调整支持的占比。

In [17]:
with open("./GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("./GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

In [18]:
full_data

[{'id': 1,
  'prompt': '日本動漫真人化是否有損原作形象？',
  'support': '真人化能夠呈現更真實的角色形象，提升原作魅力。',
  'oppose': '真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。'},
 {'id': 2,
  'prompt': '真人化是否能夠擴大動漫在全球的影響力？',
  'support': '真人化能夠讓更多非動漫迷接觸作品，擴大影響力。',
  'oppose': '真人化可能失去動漫的獨特風格，限制影響力擴大。'},
 {'id': 3,
  'prompt': '真人化是否能夠吸引新觀眾？',
  'support': '真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。',
  'oppose': '真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。'},
 {'id': 4,
  'prompt': '真人化是否能夠保留原作故事情節的精髓？',
  'support': '真人化有機會更深入挖掘原作故事，保留精髓。',
  'oppose': '真人化可能因為改編而失去原作故事的深度與精髓。'},
 {'id': 5,
  'prompt': '真人化是否能夠提升動漫產業的商業價值？',
  'support': '真人化能夠開拓更多商業機會，提升產業價值。',
  'oppose': '真人化可能讓觀眾對原作失去興趣，影響產業價值。'},
 {'id': 6,
  'prompt': '真人化是否能夠保持原作的文化特色？',
  'support': '真人化可以透過場景、服裝等元素保留文化特色。',
  'oppose': '真人化可能因為文化差異而失去原作獨有的文化魅力。'},
 {'id': 7,
  'prompt': '真人化是否能夠挑戰技術上的新突破？',
  'support': '真人化促使技術創新，挑戰視覺效果上的新高度。',
  'oppose': '真人化可能因為技術限制而無法達到動畫中的視覺效果。'},
 {'id': 8,
  'prompt': '真人化是否會受到演員選擇的爭議？',
  'support': '演員選擇可因應市場需求，不必受限於動畫形象。',
  'oppose': '演員選擇可能引起爭議，觀眾難以接受角色塑造。'},
 {'id': 9

In [19]:
test_data

[{'id': 1, 'prompt': '真人化是否能改善日本漫畫的全球可及性？'},
 {'id': 2, 'prompt': '真人化如何影響年輕一代對日本漫畫的看法？'},
 {'id': 3, 'prompt': '真人化是否能提升原作漫畫的文學價值？'},
 {'id': 4, 'prompt': '真人化是否有助於保護和保存日本漫畫的傳統？'},
 {'id': 5, 'prompt': '真人化是否有助於提升日本漫畫行業的經濟效益？'},
 {'id': 6, 'prompt': '真人化如何影響日本漫畫原作者的創作動力？'},
 {'id': 7, 'prompt': '真人化是否對漫畫原作的忠實粉絲公平？'},
 {'id': 8, 'prompt': '真人化是否能夠促進日本漫畫的創新和多樣性？'},
 {'id': 9, 'prompt': '真人化是否有助於擴大動漫文化的市場份額？'},
 {'id': 10, 'prompt': '真人化是否有助於提高日本漫畫在全球的競爭力？'}]

## 使用 HFD 下载模型

In [ ]:
# 配置选项：选择使用7B模型还是1.5B模型
USE_7B_MODEL = False  # 设置为True使用7B模型，False使用1.5B模型（推荐）
USE_4BIT_QUANTIZATION = False  # MacBook Air上4bit量化可能有兼容性问题，建议设为False

if USE_7B_MODEL:
    model_path = './Breeze-7B-Instruct-v0_1'  # 7B模型路径
    print("🚀 使用Breeze-7B-Instruct模型 + 4bit量化")
else:
    model_path = '../week06/cache/Qwen2-1.5B-Instruct'  # 1.5B模型路径
    print("🚀 使用Qwen2-1.5B-Instruct模型")

# 4bit量化配置（针对MacBook Air M4 16GB优化）
if USE_4BIT_QUANTIZATION and USE_7B_MODEL:
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,  # 使用float16而不是bfloat16
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_quant_storage=torch.uint8
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        quantization_config=quantization_config,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto",  # 自动设备映射
        max_memory={0: "14GB"},  # 限制最大内存使用
    )
    print("✅ 7B模型已加载（4bit量化）")
    
else:
    # 不使用量化的配置
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )
    print("✅ 模型已加载（无量化）")

# 设备配置（4bit量化模型的设备处理）
if USE_4BIT_QUANTIZATION and USE_7B_MODEL:
    # 4bit量化模型通常已经通过device_map="auto"自动分配设备
    print("🔧 4bit量化模型使用自动设备映射")
    device = "auto"
    
    # 检查模型实际所在的设备
    try:
        first_param_device = next(model.parameters()).device
        print(f"✅ 模型实际设备: {first_param_device}")
    except:
        print("⚠️  无法确定模型设备")
        
else:
    # 非量化模型的设备设置
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    if device == "mps":
        model = model.to('mps')
        print("✅ 使用MPS加速")
    else:
        model = model.to('cpu')
        print("🔄 使用CPU运行")
    
    print(f"模型已加载到设备: {device}")

# 显示模型信息
if USE_7B_MODEL:
    print(f"🎯 使用Breeze-7B-Instruct模型，参数量约7B")
    if USE_4BIT_QUANTIZATION:
        print(f"💾 启用4bit量化，内存占用约3.5GB")
else:
    print(f"🎯 使用Qwen2-1.5B-Instruct模型，参数量约1.5B")

✅ 使用MPS加速
模型已加载到设备: mps
✨ 使用Qwen2-1.5B-Instruct模型，参数量约1.5B（相比之前的7B模型大幅减少）


## 查看未经过微调的模型原始输出

In [ ]:
# 根据选择的模型设置tokenizer路径
if 'USE_7B_MODEL' in globals() and USE_7B_MODEL:
    tokenizer_path = './Breeze-7B-Instruct-v0_1'
    print("🔤 使用Breeze-7B tokenizer")
else:
    tokenizer_path = '../week06/cache/Qwen2-1.5B-Instruct'
    print("🔤 使用Qwen2-1.5B tokenizer")

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
print(f"✅ Tokenizer加载完成: {tokenizer_path}")

In [22]:
def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [ ]:
import time
import gc

original_model_response = []

# 确定设备（适配4bit量化模型）
if 'USE_4BIT_QUANTIZATION' in globals() and USE_4BIT_QUANTIZATION and USE_7B_MODEL:
    # 4bit量化模型使用自动设备映射
    device = "auto"
    # 获取模型实际设备
    try:
        model_device = next(model.parameters()).device
        print(f"使用设备: {model_device} (4bit量化模型)")
    except:
        model_device = "cpu"
        print("使用设备: CPU (默认)")
else:
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    model_device = device
    print(f"使用设备: {device}")

for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")
    
    start_time = time.time()
    print(f"⏳ 开始处理问题 {id}...")
    
    # 正确处理输入设备分配
    print("🔄 处理输入...")
    inputs = tokenizer(data_formulate(data), return_tensors="pt")
    
    # 根据模型类型分配输入设备
    if device == "auto":
        # 对于4bit量化模型，将输入移动到模型所在的设备
        inputs = {k: v.to(model_device) for k, v in inputs.items()}
    else:
        inputs = {k: v.to(device) for k, v in inputs.items()}
    
    print(f"✅ 输入处理完成，设备: {model_device}")
    
    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=50,  # 减少生成长度，提高速度
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id  # 添加结束token
    )
    
    print("🤖 开始生成...")
    try:
        with torch.no_grad():
            output = model.generate(**inputs, generation_config=generation_config)
        
        print("📝 解码输出...")
        output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        
        # 更安全的文本分割方式（适配Qwen2模型）
        prompt_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
        
        if 'assistant\n' in output_text:
            # Qwen2模型的输出格式
            parts = output_text.split('assistant\n')
            if len(parts) > 1:
                output_text = parts[-1].strip()
            else:
                output_text = output_text.replace(prompt_text, '').strip()
        elif '[/INST] ' in output_text:
            # 兼容其他模型的分隔符
            output_text = output_text.split('[/INST] ')[1]
        else:
            # 简单方式：移除prompt部分
            output_text = output_text.replace(prompt_text, '').strip()
        
        original_model_response.append(output_text)
        
        elapsed = time.time() - start_time
        print(f"✅ 问题 {id} 完成，耗时: {elapsed:.2f}秒")
        print(f"Response from original model:\n{output_text}\n")
        
    except Exception as e:
        print(f"❌ 生成时出错: {e}")
        original_model_response.append("生成失败")
        continue
    
    # 内存清理（适配不同设备）
    if model_device.type == "mps":
        torch.mps.empty_cache()
    elif model_device.type == "cuda":
        torch.cuda.empty_cache()
    # CPU和4bit量化模型的内存由Python GC自动管理
    
    # 每处理几个样本后清理一次内存
    if id % 3 == 0:
        gc.collect()
        print("🧹 执行内存清理")

print("🎉 所有问题处理完成！")

使用设备: mps


  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
⏳ 开始处理问题 1...
🔄 处理输入...
✅ 输入处理完成，设备: mps
🤖 开始生成...


 10%|█         | 1/10 [00:01<00:09,  1.03s/it]

📝 解码输出...
✅ 问题 1 完成，耗时: 1.03秒
Response from original model:
是的，真人化可以增加漫畫的視覺吸引力和可觀賞性。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
⏳ 开始处理问题 2...
🔄 处理输入...
✅ 输入处理完成，设备: mps
🤖 开始生成...


...

📝 解码输出...
✅ 问题 8 完成，耗时: 0.62秒
Response from original model:
是的，真人化有助於展現更多樣性。

Question 9:
真人化是否有助於擴大動漫文化的市場份額？
⏳ 开始处理问题 9...
🔄 处理输入...
✅ 输入处理完成，设备: mps
🤖 开始生成...


 90%|█████████ | 9/10 [00:07<00:00,  1.27it/s]

📝 解码输出...
✅ 问题 9 完成，耗时: 0.76秒
Response from original model:
是的，真人化可以增加動漫的吸引力和接受度。

🧹 执行内存清理
Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？
⏳ 开始处理问题 10...
🔄 处理输入...
✅ 输入处理完成，设备: mps
🤖 开始生成...


100%|██████████| 10/10 [00:08<00:00,  1.20it/s]

📝 解码输出...
✅ 问题 10 完成，耗时: 0.74秒
Response from original model:
是的，真人化可以增加漫畫的吸引力和接受度。

🎉 所有问题处理完成！


In [24]:
# 如果上面的代码仍然运行缓慢，可以尝试这个简化版本（强制使用CPU）
# 取消注释下面的代码来运行

"""
print("🔄 强制使用CPU以提高稳定性")
model = model.to("cpu")
device = "cpu"

original_model_response = []

# 先测试单个问题
test_single = test_data[0]
print(f"测试问题: {test_single['prompt']}")

inputs = tokenizer(data_formulate(test_single), return_tensors="pt")

generation_config = GenerationConfig(
    do_sample=False,
    max_new_tokens=20,  # 很短的输出
    pad_token_id=tokenizer.pad_token_id
)

print("开始生成...")
start_time = time.time()
with torch.no_grad():
    output = model.generate(**inputs, generation_config=generation_config)
print(f"生成完成，耗时: {time.time() - start_time:.2f}秒")

output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
if '[/INST] ' in output_text:
    output_text = output_text.split('[/INST] ')[1]

print(f"输出: {output_text}")
"""


'\nprint("🔄 强制使用CPU以提高稳定性")\nmodel = model.to("cpu")\ndevice = "cpu"\n\noriginal_model_response = []\n\n# 先测试单个问题\ntest_single = test_data[0]\nprint(f"测试问题: {test_single[\'prompt\']}")\n\ninputs = tokenizer(data_formulate(test_single), return_tensors="pt")\n\ngeneration_config = GenerationConfig(\n    do_sample=False,\n    max_new_tokens=20,  # 很短的输出\n    pad_token_id=tokenizer.pad_token_id\n)\n\nprint("开始生成...")\nstart_time = time.time()\nwith torch.no_grad():\n    output = model.generate(**inputs, generation_config=generation_config)\nprint(f"生成完成，耗时: {time.time() - start_time:.2f}秒")\n\noutput_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]\nif \'[/INST] \' in output_text:\n    output_text = output_text.split(\'[/INST] \')[1]\n\nprint(f"输出: {output_text}")\n'

In [25]:
# 保存结果到文件
import os
import json
from datetime import datetime

# 创建结果目录
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)

# 生成时间戳
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 准备保存的数据
results_data = {
    "model_name": "Qwen2-1.5B-Instruct",
    "model_path": "../week06/cache/Qwen2-1.5B-Instruct",
    "timestamp": timestamp,
    "device": device,
    "total_questions": len(test_data),
    "successful_responses": len([r for r in original_model_response if r != "生成失败"]),
    "results": []
}

# 组合问题和答案
for i, data in enumerate(test_data):
    result_item = {
        "id": data['id'],
        "question": data['prompt'],
        "response": original_model_response[i] if i < len(original_model_response) else "未生成",
        "status": "success" if i < len(original_model_response) and original_model_response[i] != "生成失败" else "failed"
    }
    results_data["results"].append(result_item)

# 保存为JSON文件
json_filename = f"{results_dir}/qwen2_responses_{timestamp}.json"
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(results_data, f, ensure_ascii=False, indent=2)

# 保存为文本文件（更易读）
txt_filename = f"{results_dir}/qwen2_responses_{timestamp}.txt"
with open(txt_filename, 'w', encoding='utf-8') as f:
    f.write(f"Qwen2-1.5B-Instruct 模型推理结果\n")
    f.write(f"="*50 + "\n")
    f.write(f"时间: {timestamp}\n")
    f.write(f"设备: {device}\n")
    f.write(f"模型路径: ../week06/cache/Qwen2-1.5B-Instruct\n")
    f.write(f"总问题数: {len(test_data)}\n")
    f.write(f"成功回答数: {len([r for r in original_model_response if r != '生成失败'])}\n")
    f.write(f"\n详细结果:\n")
    f.write("-"*50 + "\n")
    
    for i, data in enumerate(test_data):
        f.write(f"\n问题 {data['id']}: {data['prompt']}\n")
        response = original_model_response[i] if i < len(original_model_response) else "未生成"
        f.write(f"回答: {response}\n")
        f.write("-"*30 + "\n")

print(f"✅ 结果已保存到:")
print(f"📄 JSON格式: {json_filename}")
print(f"📝 文本格式: {txt_filename}")

# 显示保存的统计信息
print(f"\n📊 统计信息:")
print(f"- 总问题数: {len(test_data)}")
print(f"- 成功回答数: {len([r for r in original_model_response if r != '生成失败'])}")
print(f"- 使用设备: {device}")
print(f"- 模型: Qwen2-1.5B-Instruct")


✅ 结果已保存到:
📄 JSON格式: ./results/qwen2_responses_20250807_134157.json
📝 文本格式: ./results/qwen2_responses_20250807_134157.txt

📊 统计信息:
- 总问题数: 10
- 成功回答数: 10
- 使用设备: mps
- 模型: Qwen2-1.5B-Instruct


In [26]:
# DPO微调参数设置
# 只需要修改这个模块，不需要改变其他的，除非真的知道自己在做什么。

num_epoch = 3      # 训练轮数
data_size = 50      # 用于训练的数据量
support_ratio = 0.2 # 偏好支持真人化的比例

# support_ratio 将反映人类的偏好：
# 0 表示完全不支持（反对）真人化
# 1 表示完全支持真人化
# 0.1 表示 10% 支持真人化， 90% 反对。

print(f"📊 DPO训练参数:")
print(f"- 训练轮数: {num_epoch}")
print(f"- 训练数据量: {data_size}")
print(f"- 支持真人化比例: {support_ratio} ({support_ratio*100}% 支持, {(1-support_ratio)*100}% 反对)")


📊 DPO训练参数:
- 训练轮数: 3
- 训练数据量: 50
- 支持真人化比例: 0.2 (20.0% 支持, 80.0% 反对)


In [27]:
# 准备训练数据

# 选择部分数据用于训练
training_data = full_data[:data_size]

# 定义 support 数据集的大小，用于将一部分数据标记为"支持" (chosen)，另一部分标记为"反对" (rejected)
support_data_size = int(data_size * support_ratio)

print(f"📈 数据分布:")
print(f"- 总训练数据: {data_size}")
print(f"- 支持真人化的数据: {support_data_size}")
print(f"- 反对真人化的数据: {data_size - support_data_size}")

# 为训练数据集准备数据
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# 创建训练数据集
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})

# 显示数据集预览
import pandas as pd
df_preview = pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})
print(f"\n📋 训练数据集预览 (前5行):")
print(df_preview[['position', 'preferred', 'non-preferred']].head())

print(f"\n✅ 训练数据集准备完成!")
print(f"- 数据集大小: {len(train_dataset)}")
print(f"- 总共有 {data_size} 笔训练数据")
print(f"- 当 support_ratio 设置为 {support_ratio} 时:")
print(f"  * 前 {support_data_size} 笔数据偏好支持真人化")
print(f"  * 后 {data_size - support_data_size} 笔数据偏好反对真人化")


📈 数据分布:
- 总训练数据: 50
- 支持真人化的数据: 10
- 反对真人化的数据: 40

📋 训练数据集预览 (前5行):
  position                preferred                non-preferred
0  support  真人化能夠呈現更真實的角色形象，提升原作魅力。  真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。
1  support  真人化能夠讓更多非動漫迷接觸作品，擴大影響力。      真人化可能失去動漫的獨特風格，限制影響力擴大。
2  support    真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。    真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。
3  support    真人化有機會更深入挖掘原作故事，保留精髓。      真人化可能因為改編而失去原作故事的深度與精髓。
4  support    真人化能夠開拓更多商業機會，提升產業價值。      真人化可能讓觀眾對原作失去興趣，影響產業價值。

✅ 训练数据集准备完成!
- 数据集大小: 50
- 总共有 50 笔训练数据
- 当 support_ratio 设置为 0.2 时:
  * 前 10 笔数据偏好支持真人化
  * 后 40 笔数据偏好反对真人化


In [28]:
# 训练配置

print("🔧 设置训练参数...")

# 确保使用正确的设备
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"训练设备: {device}")

# DPO训练配置 - 针对MacBook Air M4优化
training_args = DPOConfig(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="adamw_torch",  # 使用标准AdamW优化器，避免8bit问题
    logging_steps=1,
    warmup_ratio=0.1,
    beta=0.1,
    report_to='none',
    
    # 显式声明以避免警告
    max_length=512,
    max_prompt_length=128,
    remove_unused_columns=False,
    
    # MacBook Air M4兼容性设置
    fp16=False,  # 禁用fp16
    bf16=False,  # 禁用bf16
    dataloader_pin_memory=False,  # 禁用内存固定
    use_cpu=True if device == "cpu" else False,  # CPU模式
)

print("✅ DPO训练参数设置完成")

# PEFT配置 - Parameter-Efficient Fine-Tuning
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

print("✅ PEFT (LoRA) 配置完成")

print(f"\n📋 训练配置总览:")
print(f"- 训练轮数: {num_epoch}")
print(f"- 批大小: {training_args.per_device_train_batch_size}")
print(f"- 梯度累积步数: {training_args.gradient_accumulation_steps}")
print(f"- 学习率: {training_args.learning_rate}")
print(f"- LoRA rank: {peft_config.r}")
print(f"- LoRA alpha: {peft_config.lora_alpha}")


🔧 设置训练参数...
训练设备: mps
✅ DPO训练参数设置完成
✅ PEFT (LoRA) 配置完成

📋 训练配置总览:
- 训练轮数: 3
- 批大小: 1
- 梯度累积步数: 8
- 学习率: 0.0002
- LoRA rank: 64
- LoRA alpha: 16


In [29]:
# 初始化DPO训练器并开始训练

print("🚀 初始化DPO训练器...")

# 确保模型在正确的设备上
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"将模型移动到设备: {device}")

# 如果使用CPU训练，将模型移动到CPU
if device == "cpu":
    model = model.to("cpu")
    print("✅ 模型已移动到CPU")
elif device == "mps":
    # MPS可能在训练时有兼容性问题，建议使用CPU
    print("⚠️  MPS在DPO训练时可能不稳定，建议使用CPU")
    model = model.to("cpu")
    device = "cpu"
    print("✅ 已切换到CPU进行训练")

# 初始化 DPO 训练器
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

print("✅ DPO训练器初始化完成")

print(f"\n🔥 开始DPO训练...")
print(f"- 训练数据量: {len(train_dataset)}")
print(f"- 训练轮数: {num_epoch}")
print(f"- 支持比例: {support_ratio}")

# 开始训练
import time
train_start_time = time.time()

try:
    dpo_trainer.train()
    train_elapsed = time.time() - train_start_time
    print(f"\n🎉 DPO训练完成!")
    print(f"- 训练耗时: {train_elapsed:.2f}秒")
    print(f"- 平均每轮: {train_elapsed/num_epoch:.2f}秒")
    
except Exception as e:
    print(f"\n❌ 训练过程中出现错误: {e}")
    import traceback
    traceback.print_exc()


🚀 初始化DPO训练器...
将模型移动到设备: mps
⚠️  MPS在DPO训练时可能不稳定，建议使用CPU
✅ 已切换到CPU进行训练


Tokenizing train dataset: 100%|██████████| 50/50 [00:00<00:00, 4502.94 examples/s]


✅ DPO训练器初始化完成

🔥 开始DPO训练...
- 训练数据量: 50
- 训练轮数: 3
- 支持比例: 0.2


Step,Training Loss
1,0.692600
2,0.695200
3,0.690800
4,0.686900
5,0.674700
6,0.643400
7,0.699700
8,0.601100
9,0.592000
10,0.545900



🎉 DPO训练完成!
- 训练耗时: 177.67秒
- 平均每轮: 59.22秒


In [30]:
# 测试训练后的模型

print("🧪 测试训练后的模型效果...")

trained_model_response = []

# 确定设备
device = "mps" if torch.backends.mps.is_available() else "cpu"

for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    # 处理输入 - 适配Qwen2和MPS设备
    inputs = tokenizer(data_formulate(data), return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=50,  # 保持与原始测试一致的长度
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    
    try:
        with torch.no_grad():
            output = model.generate(**inputs, generation_config=generation_config)
        
        output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        prompt_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
        
        # 提取回答 - 适配Qwen2模型格式
        if 'assistant' in output_text:
            parts = output_text.split('assistant')
            if len(parts) > 1:
                response = parts[-1].strip()
            else:
                response = output_text.replace(prompt_text, '').strip()
        else:
            response = output_text.replace(prompt_text, '').strip()
        
        trained_model_response.append(response)
        print(f"Response from trained model:\n{response}\n")
        
    except Exception as e:
        print(f"❌ 生成失败: {e}")
        trained_model_response.append("生成失败")
    
    # 内存清理
    if device == "mps":
        torch.mps.empty_cache()

print(f"✅ 训练后模型测试完成!")
print(f"- 成功回答: {len([r for r in trained_model_response if r != '生成失败'])}/{len(test_data)}")


🧪 测试训练后的模型效果...


  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？


 10%|█         | 1/10 [00:02<00:26,  2.91s/it]

Response from trained model:
是的，真人化可以增加漫畫的吸引力和接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？


...

Response from trained model:
是的，真人化有助於更多元化和創新。

Question 9:
真人化是否有助於擴大動漫文化的市場份額？


 90%|█████████ | 9/10 [00:22<00:02,  2.39s/it]

Response from trained model:
是的，真人化可以增加觀眾吸引力。

Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？


100%|██████████| 10/10 [00:25<00:00,  2.52s/it]

Response from trained model:
是的，真人化可以增加觀眾的吸引力。

✅ 训练后模型测试完成!
- 成功回答: 10/10


In [31]:
# 对比训练前后的结果并保存

print("📊 对比训练前后的模型回答...")

# 创建对比数据
comparison_data = {
    "model_name": "Qwen2-1.5B-Instruct",
    "model_path": "../week06/cache/Qwen2-1.5B-Instruct",
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "training_params": {
        "num_epoch": num_epoch,
        "data_size": data_size,
        "support_ratio": support_ratio
    },
    "device": device,
    "comparisons": []
}

print(f"\n🔍 详细对比结果:")
print("="*80)

for i, data in enumerate(test_data):
    original_resp = original_model_response[i] if i < len(original_model_response) else "未生成"
    trained_resp = trained_model_response[i] if i < len(trained_model_response) else "未生成"
    
    comparison_item = {
        "id": data['id'],
        "question": data['prompt'],
        "original_response": original_resp,
        "trained_response": trained_resp
    }
    comparison_data["comparisons"].append(comparison_item)
    
    print(f"\n问题 {data['id']}: {data['prompt']}")
    print(f"原始模型: {original_resp}")
    print(f"训练后模型: {trained_resp}")
    print("-"*60)

# 保存对比结果
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
comparison_filename = f"{results_dir}/dpo_comparison_{timestamp}.json"

with open(comparison_filename, 'w', encoding='utf-8') as f:
    json.dump(comparison_data, f, ensure_ascii=False, indent=2)

# 保存详细的文本报告
report_filename = f"{results_dir}/dpo_training_report_{timestamp}.txt"
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write(f"DPO训练报告 - Qwen2-1.5B-Instruct\n")
    f.write(f"="*50 + "\n")
    f.write(f"时间: {timestamp}\n")
    f.write(f"模型: Qwen2-1.5B-Instruct\n")
    f.write(f"设备: {device}\n\n")
    
    f.write(f"训练参数:\n")
    f.write(f"- 训练轮数: {num_epoch}\n")
    f.write(f"- 训练数据量: {data_size}\n")
    f.write(f"- 支持真人化比例: {support_ratio}\n\n")
    
    f.write(f"结果对比:\n")
    f.write(f"-"*50 + "\n")
    
    for i, data in enumerate(test_data):
        original_resp = original_model_response[i] if i < len(original_model_response) else "未生成"
        trained_resp = trained_model_response[i] if i < len(trained_model_response) else "未生成"
        
        f.write(f"\n问题 {data['id']}: {data['prompt']}\n")
        f.write(f"原始模型: {original_resp}\n")
        f.write(f"训练后模型: {trained_resp}\n")
        f.write(f"-"*30 + "\n")

print(f"\n✅ 结果已保存:")
print(f"📄 对比数据 (JSON): {comparison_filename}")
print(f"📝 训练报告 (TXT): {report_filename}")

print(f"\n📈 训练总结:")
print(f"- 使用模型: Qwen2-1.5B-Instruct")
print(f"- 训练方法: DPO (Direct Preference Optimization)")
print(f"- 训练数据: {data_size} 条")
print(f"- 偏好设置: {support_ratio*100}% 支持真人化")
print(f"- 测试问题: {len(test_data)} 个")
print(f"- 设备: {device}")



📊 对比训练前后的模型回答...

🔍 详细对比结果:

问题 1: 真人化是否能改善日本漫畫的全球可及性？
原始模型: 是的，真人化可以增加漫畫的視覺吸引力和可觀賞性。
训练后模型: 是的，真人化可以增加漫畫的吸引力和接受度。
------------------------------------------------------------

问题 2: 真人化如何影響年輕一代對日本漫畫的看法？
原始模型: 真人化使漫畫更接近生活，吸引年輕觀眾。
训练后模型: 真人化增加漫畫知名度，但可能淡化原作深度。
------------------------------------------------------------

问题 3: 真人化是否能提升原作漫畫的文學價值？
原始模型: 真人化可以增加觀眾的接觸度，但不一定提升原作的文學價值。
训练后模型: 不一定，視情況而定。
------------------------------------------------------------

问题 4: 真人化是否有助於保護和保存日本漫畫的傳統？
原始模型: 是的，真人化有助於保持漫畫的傳統風格和故事性。
训练后模型: 是的，真人化有助於保持原汁原味。
------------------------------------------------------------

问题 5: 真人化是否有助於提升日本漫畫行業的經濟效益？
原始模型: 是的，真人化可以增加觀眾吸引力和收視率。
训练后模型: 是的，真人化可以增加收視率和票房收益。
------------------------------------------------------------

问题 6: 真人化如何影響日本漫畫原作者的創作動力？
原始模型: 真人化增加創作靈感，但可能影響創作風格。
训练后模型: 真人化可能降低創作動力。
------------------------------------------------------------

问题 7: 真人化是否對漫畫原作的忠實粉絲公平？
原始模型: 真人化可能影響原作的深度和細節。
训练后模型: 真人化可能削弱原作的深度和深度。
------------------------------------------